In [1]:
import models
import pandas as pd
from sklearn.pipeline import make_pipeline
from preprocess import TypeConverter, KBinsDiscretizerWithNames, OneHotEncoderOnlyCategorical
from splitter import StratifiedRegressionSplit
from sklearn.metrics import r2_score
import numpy as np

In [2]:
variables = pd.read_excel('../data/covariates.xlsx')
data = pd.read_excel('../data/hp_ljubljana_new_with_rooms.xlsx')

variables = variables.to_dict(orient='records')

target = []
features = []
for feature in variables:
    feature['type'] = int if feature['type'] == 'int' else float if feature['type'] == 'float' else str
    if feature['group'] == 'target':
        target = feature
        continue
    else:
        features.append(feature)

X = data[[feature['name'] for feature in features]]
y = data[target['name']]

# PREPROCESS PIPLINE
preprocc_pip = make_pipeline(TypeConverter(variables=features), KBinsDiscretizerWithNames(
                            encode='onehot-dense',
                            strategy='quantile',
                            random_state=0,
                            n_bins=2),
            OneHotEncoderOnlyCategorical())
preprocc_pip.fit(X)
X = preprocc_pip.transform(X)




/home/sjelic/real_estate_prices/.venv/lib/python3.8/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 22 are removed. Consider decreasing the number of bins.
  warnings.warn(


In [3]:
categorical_features = { n: t for n, t in X.dtypes.to_dict().items() if t in [ np.dtype('O') ]}
numerical_features = { n: t for n, t in X.dtypes.to_dict().items() if t not in [ np.dtype('O') ]}
cat_cols = list(categorical_features.keys())
num_cols = list(numerical_features.keys())
bin_num_cols = []
nonbin_num_cols = []
for num_col in num_cols:
    is_bin = True
    for val in X[num_col]:
        if (np.abs(1-val) > 1e-3) and (np.abs(val) > 1e-3):
            is_bin = False
    
    if is_bin:
        bin_num_cols.append(num_col)
    else:
        nonbin_num_cols.append(num_col)

binary_numerical_features = { n: t for n, t in X.dtypes.to_dict().items() if n in bin_num_cols}
nonbinary_numerical_features = { n: t for n, t in X.dtypes.to_dict().items() if n in nonbin_num_cols}

feature_names_in_ = np.array(X.columns, dtype=object)
n_features_in_ = len(feature_names_in_)
# return self

In [6]:
bin_num_cols

['dist_airport_[11175.8671875, 18748.06640625)',
 'dist_airport_[18748.06640625, 25051.1484375)',
 'dist_highway_entr_[100.0, 1431.78210449219)',
 'dist_highway_entr_[1431.78210449219, 4341.65869140625)',
 'dist_main_roads_[0.0, 200.0)',
 'dist_main_roads_[200.0, 4110.9609375)',
 'dist_public_transport_[0.0, 200.0)',
 'dist_public_transport_[200.0, 4134.00537109375)',
 'dist_recreation_[100.0, 1044.03063964844)',
 'dist_recreation_[1044.03063964844, 5280.1513671875)',
 'dist_regional_roads_[0.0, 2061.552734375)',
 'dist_regional_roads_[2061.552734375, 3676.95532226562)',
 'dist_river_[0.0, 583.09521484375)',
 'dist_river_[583.09521484375, 2308.67919921875)',
 'dist_schools_[0.0, 200.0)',
 'dist_schools_[200.0, 3600.0)',
 'construct_age_[0.0, 13327.0)',
 'construct_age_[13327.0, 74325.0)',
 'facade_age_[0.0, 11792.0)',
 'facade_age_[11792.0, 66580.0)',
 'inst_age_[0.0, 11912.0)',
 'inst_age_[11912.0, 67040.0)',
 'roof_age_[0.0, 7603.0)',
 'roof_age_[7603.0, 66580.0)',
 'windows_age_[0.0